In [1]:
import pandas as pd
import dill
import missingno as msno
import matplotlib.pyplot as plt



# Загрузил df_hits
df_hits = pd.read_csv('data/ga_hits.csv').drop_duplicates()
# Загрузил df_sessions
df_sessions = pd.read_csv('data/ga_sessions.csv')


C:\Users\Mike\AppData\Local\Temp\ipykernel_6780\2455362022.py:11: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sessions = pd.read_csv('data/ga_sessions.csv')


In [ ]:
msno.matrix(df_hits)

In [ ]:
msno.matrix(df_sessions)

In [ ]:
df_hits.isna().sum()

In [ ]:
df_sessions.isna().sum()

In [ ]:
df_hits.hit_time.describe()

In [ ]:
!p

In [ ]:
import pandas_profiling

In [ ]:
pandas_profiling.ProfileReport(df_hits)

In [ ]:
pandas_profiling.ProfileReport(df_sessions)

In [ ]:
# Добавил столбец target_action 1 - целевое действие - иначе 0
target_action = [
    'sub_car_claim_click',
    'sub_car_claim_submit_click',
    'sub_open_dialog_click',
    'sub_custom_question_submit_click',
    'sub_call_number_click',
    'sub_callback_submit_click',
    'sub_submit_success',
    'sub_car_request_submit_click']
df_hits['target_action'] = df_hits.event_action.apply(lambda x: 1 if x in target_action else 0)


# Удалил пустой столбец
df_hits = df_hits.drop('event_value', axis = 1)

# Вместо адреса добавил 2 столбца manufacturer and model.
#df_hits[['manufacturer', 'model']] = df_hits.hit_page_path.str.split('/', expand=True)[[3, 4]]
# Столбец hit page path удалил. Строки с пропусками...

df_hits = df_hits.drop(['hit_time', 'hit_referer', 'event_label', 'hit_page_path', 'event_action'], axis=1)

# в Dataframe res хранится id сессии и факт выполнения целевого действия
res = df_hits[['session_id', 'target_action']].groupby(by='session_id').sum()
res.target_action = res.target_action.apply(lambda x: 1 if x > 0 else 0)
res.reset_index()

In [ ]:
# Дропаем
df_sessions = df_sessions.drop(['device_model', 'utm_keyword'], axis=1)

# Заполняю utm_source значением other
columns = ['utm_source', 'utm_campaign', 'device_os', 'device_brand', 'utm_adcontent']
df_sessions[columns] = df_sessions[columns].fillna('other')

In [ ]:
df_sessions